<a href="https://colab.research.google.com/github/Charles-Scott-Green/Argument-mining/blob/master/4_Clean_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!python -m spacy download en_core_web_lg --quiet

     |████████████████████████████████| 826.9MB 5.8MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.8MB/s 


In [0]:
!pip install spacy-wordnet

     |████████████████████████████████| 655kB 2.8MB/s 
     |████████████████████████████████| 1.4MB 13.7MB/s 
  Created wheel for spacy-wordnet: filename=spacy_wordnet-0.0.4-py2.py3-none-any.whl size=650292 sha256=2f57673f47da0c2565eedb87acf07735dc0b76eaf6e34cc643e05a6001583f2e
  Stored in directory: /root/.cache/pip/wheels/25/93/1d/c86db913cd146fc9ddb26d10f56579c5d58a3e00bc8f96a3a6
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394470 sha256=4aeb5e0b757b2b41815e1ed6b6f8b92010ec560557d604046d76395a1d55b948
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built spacy-wordnet nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
!pip install pyLanguagetool

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 40kB 7.0MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.0-cp36-none-any.whl size=17874 sha256=aa7c5b2ba9c562ef1e5720183ecede6e8f69d905cadd28c776f9144ea6e35bf0
  Stored in directory: /root/.cache/pip/wheels/0d/3a/3f/d9c3d44d4e15f125c92bd64b611106937511e20ecc34bb5229
Successfully built configargparse
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0


## Import Libraries and initialize settings

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [0]:
import re
import string
import math
from tqdm import tqdm

In [0]:
import nltk
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize, word_tokenize
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [0]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

In [0]:
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
from spacy.lang.en.stop_words import STOP_WORDS
from spacy_wordnet.wordnet_annotator import WordnetAnnotator

nlp = spacy.load("en_core_web_lg")
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')

In [0]:
import warnings
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
warnings.filterwarnings('ignore')

In [0]:
from sklearn.model_selection import train_test_split as tts 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb

from imblearn.over_sampling import SMOTE

In [0]:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pickle

In [0]:
import pylanguagetool
from pylanguagetool.api import check as chkr

## Define Functions

In [0]:
def get_target_data(path):
    df = pd.read_csv(path)
    df['cdc_present'] = (df['cdc'] != '---').astype(int)

    return df['cdc_present']

In [0]:
def topic_domains(topic):
    raw_domains = []
    common_domains = []

    for chunk in nlp(topic).noun_chunks:
        rt_nn = chunk.root
        raw_domains.append(rt_nn._.wordnet.wordnet_domains())

    if len(raw_domains) > 0:
        reference = raw_domains[0]
    else:
        return list(raw_domains)
        
    for domain in raw_domains:
        common_domains = set(reference) & set(domain)
        
    return list(common_domains)

In [0]:
def expanded_sentence_similarity(topic, sentence):
    
    extended_topic = []
    domains = topic_domains(topic)
    
    if len(domains) < 1:
        return nlp(topic).similarity(nlp(sentence))
    else:
        for token in nlp(topic):
            synsets = token._.wordnet.wordnet_synsets_for_domain(domains)
            if synsets: 
                lfs = []
                for syn in synsets:
                    lfs.extend(syn.lemma_names())
                    extended_topic.append('({})'.format('|'.join(set(lfs))))
            else:
                extended_topic.append(token.text)

        tokenized_extended_topic = nlp(' '.join(extended_topic))

    return tokenized_extended_topic.similarity(nlp(sentence))            

In [0]:
def subject_match(topic, sentence):
    
    ntopic = nlp(topic)
    scores = []
    for chunk in nlp(sentence).noun_chunks:
        scores.append(chunk.similarity(ntopic))
    
    return np.mean(scores)

In [0]:
def subject_match2(topic, sentence):
    
    ntopic = nlp(topic)
    scores = []
    for chnk in nlp(sentence).noun_chunks:
        for tchnk in ntopic.noun_chunks:
            scores.append(chnk.similarity(tchnk))
    
    return np.mean(scores)

In [0]:
def includes_that(sentence):
    check = []
    for tok in sentence:
        if tok.tag_ == 'IN':
            check.append(tok.text)
    if 'that' in check:
        return 1
    else:
        return 0

In [0]:
def patterns(sentence, pattern_type='dep'):
    
    pat = []
    if pattern_type == 'dep':
        pat = [tok.dep_ for tok in sentence]
    elif pattern_type == 'tag':
        pat = [tok.tag_ for tok in sentence]
    
    return '|'.join(pat)

In [0]:
def pos_pattern(sentence):
    pat = []
    for tok in sentence:
        pat.append(tok.pos_)

    return '|'.join(pat)


In [0]:
def pattern_in_text(pattern, sentence):
    text_pattern = pos_pattern(sentence)

    if pattern in text_pattern:
        return 1
    else:
        return 0

In [0]:
def infinitive_verb_present(sentence):
    text_pattern = pos_pattern(sentence)
    infinitive_pattern = 'PART|VERB'

    if infinitive_pattern in text_pattern:
        return 1
    else:
        return 0


In [0]:
def sentiment_subjectivity(sentence, score_type='sentiment'):
    analyzer = SentimentIntensityAnalyzer()

    if score_type == 'sentiment':
        pos = analyzer.polarity_scores(sentence)['pos']
        neg = analyzer.polarity_scores(sentence)['neg']
        return pos/neg
    
    else:
        return analyzer.polarity_scores(sentence)['compound']

In [0]:
def is_sentence(clause):
    nn = 2
    vb = 1
    cls = nlp(clause)
    for tok in cls:
        if tok.pos_ in ['NOUN', 'PROPN', 'PRON']:
            nn -= 1
        elif tok.pos_ in ['VERB']:
            vb -= 1
    if nn < 1 and vb < 1:
        return True
    else:
        return False

In [0]:
def get_clauses(sentence):
    """Sentence type is a string."""
    subtree_phrases = []
    
    for tok in nlp(sentence):
        if tok.head.pos_ == 'VERB':
            if len([tok for tok in tok.subtree]) >= 3:
                subtree_prhase = ''.join(tok.text_with_ws for tok in tok.subtree)
                if subtree_prhase != sentence:
                    if is_sentence(subtree_prhase):
                        splits = subtree_prhase.split(',')
                        if len(splits) == 1:
                            subtree_phrases.append(subtree_prhase)
                        else:
                            for split in splits:
                                if is_sentence(split):
                                    if split not in subtree_phrases:
                                        subtree_phrases.append(split) 
    
    raw_clauses = []
    clauses = []

    for phrase in subtree_phrases:
        tok_phrase = nlp(phrase)
        for chunk in tok_phrase.noun_chunks:
            span = tok_phrase[chunk.root.head.left_edge.i:chunk.root.head.right_edge.i+1]
            if span[0].pos_ in ['DET', 'ADP']:
                if span[1:].text not in raw_clauses:
                    if is_sentence(span[1:].text):
                        raw_clauses.append(span[1:].text)
            elif span.text not in raw_clauses:
                if is_sentence(span.text):
                    raw_clauses.append(span.text)
    
    if len(raw_clauses) == 0:
        clauses.append(sentence)
    else:
        for clause in raw_clauses:
            dup_checks = [(clause in check_clause) for check_clause in raw_clauses]
            if sum(dup_checks) == 1:
                clauses.append(clause)
                
    return clauses

In [0]:
def conf_matrix(y_true, y_predict):
    
    from sklearn.metrics import confusion_matrix
    
    data = confusion_matrix(y_true, y_predict)
    index = ['Actual_0', 'Actual_1']
    columns = ['Predicted_0', 'Predicted_1']
    
    return pd.DataFrame(data, index, columns)

In [0]:
def get_target_data(path):
    df = pd.read_csv(path)
    df['cdc_present'] = (df['cdc'] != '---').astype(int)

    return df['cdc_present']

In [0]:
def count_sentiment_words(sentence):
    analyzer = SentimentIntensityAnalyzer()
    wd_cnt = 0

    for tok in nlp(sentence):
        if analyzer.polarity_scores(tok.text)['compound'] != 0:
            wd_cnt += 1

    return wd_cnt

In [0]:
def count_syllables(sentence):
    """Given a string, calculates and returns the number of syllables of the word."""
    import string
    
    word_list = sentence.translate(string.punctuation).split()
    vowels = "aeiouy"
    tally = 0
    
    for word in word_list:
        word = word.lower()
        vowels = "aeiouy"
        count = 0
        
        if word[0] in vowels:
            count += 1

        for i in range(1, len(word)):
            if word[i] in vowels and word[i - 1] not in vowels:
                count += 1

        if word.endswith('e'):
            count -= 1

        if count == 0:
            count += 1
    
        tally += count
    
    return tally

In [0]:
def punct_marks(sentence):
    """Given a sentence, returns count and/or list of punctiation marks"""
    
    pm = []
    
    for tok in nlp(sentence):
        if tok.pos_ == 'PUNCT':
            pm.append(tok)
    return len(pm)

In [0]:
def noun_counts(sentence):
    
    chnk = []
    
    for chunk in nlp(sentence).noun_chunks:
        chnk.append(chunk)

    return len(chnk)

In [0]:
def word_count(sentence):
    wds = []
    for tok in nlp(sentence):
        if tok.pos_ != 'PUNCT':
            wds.append(tok)
    return len(wds)

In [0]:
def phrase_chunks(sentence):
    s = []
    for tok in nlp(sentence):
        if len([tok for tok in tok.subtree]) >=3:
            if ''.join(tok.text_with_ws for tok in tok.subtree) != str(sentence):
                s.append(''.join(tok.text_with_ws for tok in tok.subtree))
    return len(s)

In [0]:
def text_cleaner(text):
    # visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  better get rid of it now!
    text = text.strip()
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = text.replace(' .', '.')
    text = text.replace('/', '')
    text = re.sub("\'s", "'s", text)
    text = ' '.join(text.split())

    return text

In [0]:
def sentencify(clause):
    clause = text_cleaner(clause)
    if clause[-1] == '.':
        clause = clause[0].upper() + clause[1:]
    else:
        clause = clause[0].upper() + clause[1:] + '.'
    
    return clause

In [0]:
def exclude_rule(flags):
    skip_messages = ['Use a smart closing quote here: "”".', 
                     "Don't put a space after the opening parenthesis", 
                     'Use a smart opening quote here: "“".', 
                     "Don't put a space before the closing parenthesis", 
                     'Possible spelling mistake', 
                     'Two consecutive dots']

    rule_count = []

    for flag in flags:
        for x in skip_messages:
            if x in flag:
                rule_count.append(1)
            else:
                rule_count.append(0)
    
    if sum(rule_count) == len(flags): 
        return True
    else:
        return False

def check_grammar(clause):
    
    if clause == '.':
        return False
    else:
        n_clause = sentencify(clause)

        good_grammar = chkr(n_clause, api_url='https://languagetool.org/api/v2/', lang='en-US')['matches']
        
        if len(good_grammar) == 0 and is_sentence(clause):
            return True
        else:
            messages = [m['message'] for m in good_grammar]
            if exclude_rule(messages):
                return True
            else:
                return False
    


In [0]:
def calculate_stance(topic, claim):
    topic = topic + 'is the best'
    
    if claim == '.':
        
        return 0
    
    else:
        p_claim = analyzer.polarity_scores(claim)['compound']
        p_topic = analyzer.polarity_scores(topic)['compound']
        rel_sent = subject_match(topic, claim)
    
        return p_claim * rel_sent * p_topic

In [0]:
def trim_dataframe(df):
    cls = [2,3,4,5,6]
    grm = [7,8,9,10,11]
    stn = [12,13,14,15,16]
    cols = ['cdc_present', 'context', 
            'clause', 'stance']
    
    ndf = df.copy()
    stances = []
    clauses = []

    for x in df.index:
        gr_chk = list(df.iloc[x,grm])
        gd_gram = np.nonzero(gr_chk)

        if len(gd_gram[0]) == 0:
            clauses.append('---') 
            stances.append(0)
        else:
            scores = list(abs(df.iloc[x, stn]))
            hi_scr = max(abs(df.iloc[x,stn]))
            sc_idx = scores.index(hi_scr)
            if sc_idx in gd_gram[0]:
                clauses.append(df.iloc[x,cls[sc_idx]])
                stances.append(df.iloc[x,stn[sc_idx]])
            else:
                idx = gd_gram[0][0]
                clauses.append(df.iloc[i,cls[idx]])
                stances.append(df.iloc[i,stn[idx]])
                

    ndf['clause'] = clauses
    ndf['stance'] = stances
    ndf = ndf.loc[ndf['clause'] != '---']
    ndf = ndf[cols]
    ndf.reset_index(drop=True, inplace=True)

    return ndf

def trim_test_dataframe(df, cols):
    cls = [19,20,21,22,23]
    grm = [24,25,26,27,28]
    stn = [29,30,31,32,33]
    
    ndf = df.copy()
    stances = []
    clauses = []

    for x in df.index:
        gr_chk = list(df.iloc[x,grm])
        gd_gram = np.nonzero(gr_chk)

        if len(gd_gram[0]) == 0:
            clauses.append('---') 
            stances.append(0)
        else:
            scores = list(abs(df.iloc[x, stn]))
            hi_scr = max(abs(df.iloc[x,stn]))
            sc_idx = scores.index(hi_scr)
            if sc_idx in gd_gram[0]:
                clauses.append(df.iloc[x,cls[sc_idx]])
                stances.append(df.iloc[x,stn[sc_idx]])
            else:
                idx = gd_gram[0][0]
                clauses.append(df.iloc[i,cls[idx]])
                stances.append(df.iloc[i,stn[idx]])
                

    ndf['clause'] = clauses
    ndf['stance'] = stances
    ndf = ndf.loc[ndf['clause'] != '---']
    ndf = ndf[cols]
    ndf.reset_index(drop=True, inplace=True)

    return ndf

## Load Dataframe & Model

In [0]:
# paths to data
path1 = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/cleaned.csv'
path2 = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/clause_sentences.csv'
path3 = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/funnel_1_model.sav'
path4 = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/funnel_1.csv'

train_data = pd.read_csv(path4)
raw_data = pd.read_csv(path1)
cls_data = pd.read_csv(path2)

In [0]:
raw_data.shape, cls_data.shape, train_data.shape

((41999, 21), (41999, 6), (10500, 302))

In [0]:
df = pd.concat([raw_data, cls_data], axis=1)
df.head()

,Unnamed: 0,Unnamed: 0.1,text,topic,article,cdc,cdc_present,context,text_words,text_syllables,text_sentiment_words,text_punctuation,text_nouns,text_phrases,sent_rat,clauses,num_clauses,claused_sentence,context_similarity,expanded_similarity,subjectivity_score,Unnamed: 0,clause_0,clause_1,clause_2,clause_3,clause_4
0,0,0,Deficit spending is the amount by which a gove...,Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,35,67,3,10,10,12,0.085714,"["" or individual's spending exceeds income ove...",1,or individual's spending exceeds income over ...,0.326836,0.259504,0.0000,0,or individual's spending exceeds income over ...,.,.,.,.
1,1,1,Government deficit spending is a central point...,Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,14,29,1,2,4,5,0.071429,['Government deficit spending is a central poi...,1,Government deficit spending is a central point...,0.373849,0.267986,-0.4019,1,Government deficit spending is a central point...,.,.,.,.
2,2,2,Government deficit spending is a central point...,Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,17,37,2,2,6,6,0.117647,['prominent economists holding differing views'],1,prominent economists holding differing views,0.267243,0.106333,0.3182,2,prominent economists holding differing views,.,.,.,.
3,3,3,The mainstream economics position is that defi...,Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,66,122,7,7,15,22,0.106061,['deficit spending is desirable and necessary ...,4,deficit spending is desirable and necessary as...,0.461772,0.240641,-0.9280,3,deficit spending is desirable and necessary as...,but that there should not be a structural def...,government should run deficits,but should run corresponding surpluses in boo...,.
4,4,4,"This is derived from Keynesian economics, and ...",Europe should weaken its austerity measures to...,Deficit spending,---,False,austerity measures,34,64,3,5,6,11,0.088235,['has been the mainstream economics view (in t...,1,has been the mainstream economics view (in the...,0.318571,0.306673,0.3612,4,has been the mainstream economics view (in the...,.,.,.,.


In [0]:
print(df['text'][18])

Chartalists argue that deficit spending is logically necessary because, in their view, fiat money is created by deficit spending: one cannot collect fiat money in taxes before one has issued it and spent it, and the amount of fiat money in circulation is exactly the government debt – money spent but not collected in taxes.


In [0]:
print(df['clause_0'][18])
print(df['clause_1'][18])
print(df['clause_2'][18])
print(df['clause_3'][18])

Chartalists argue that deficit spending is logically necessary because
 fiat money is created by deficit spending
one has issued it and spent it
amount of fiat money in circulation is exactly the government debt – money spent but not collected in taxes.


In [0]:
displacy.render(nlp('This presentation is awesome!'), jupyter=True,)

In [0]:
displacy.render(nlp(df['cdc'][18]), jupyter=True,
                options={'compact':True, 'distance':100})

In [0]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [0]:
df.columns

Index(['text', 'topic', 'article', 'cdc', 'cdc_present', 'context',
       'text_words', 'text_syllables', 'text_sentiment_words',
       'text_punctuation', 'text_nouns', 'text_phrases', 'sent_rat', 'clauses',
       'num_clauses', 'claused_sentence', 'context_similarity',
       'expanded_similarity', 'subjectivity_score', 'clause_0', 'clause_1',
       'clause_2', 'clause_3', 'clause_4'],
      dtype='object')

### Preprocessing

In [0]:
train_data.head()

,Unnamed: 0,txt_0,txt_1,txt_2,txt_3,txt_4,txt_5,txt_6,txt_7,txt_8,txt_10,txt_11,txt_12,txt_13,txt_14,txt_15,txt_16,txt_18,txt_19,txt_20,txt_21,txt_22,txt_23,txt_24,txt_25,txt_26,txt_27,txt_28,txt_29,txt_30,txt_31,txt_32,txt_33,txt_34,txt_35,txt_36,txt_37,txt_38,txt_39,txt_40,...,txt_267,txt_271,txt_272,txt_273,txt_274,txt_275,txt_276,txt_277,txt_278,txt_279,txt_280,txt_281,txt_282,txt_283,txt_284,txt_285,txt_286,txt_287,txt_288,txt_289,txt_290,txt_291,txt_292,txt_293,txt_294,txt_295,txt_296,txt_297,txt_298,txt_299,0,1,2,3,4,5,6,7,8,cdc_present
0,0,-0.043492,0.255405,-0.031507,-0.062282,0.063146,-0.003994,-0.055570,-0.098684,-0.005464,-0.296168,0.114405,0.087672,0.035144,-0.041646,-0.125251,0.001274,-0.164186,0.085831,-0.037350,-0.019712,-0.124259,-0.046066,-0.035945,-0.025597,-0.090412,0.030220,-0.018963,-0.066032,-0.014213,0.064528,-0.089806,0.122765,0.135511,-0.083966,-0.071495,-0.004060,-0.063389,-0.081253,0.067016,...,-0.145454,-0.138180,-0.011381,-0.247776,0.002634,-0.089299,-0.033787,0.065599,0.097083,0.118096,0.180134,0.026501,0.009148,0.085887,-0.012560,-0.017416,-0.016740,-0.146287,0.061698,0.034576,-0.177744,0.141555,0.100658,0.044665,-0.017619,0.002235,-0.118059,-0.151709,-0.048323,-0.083152,0.787621,0.976035,0.207532,2.286923,0.900853,0.524713,-0.108448,-0.258628,-0.019056,False
1,1,-0.109559,0.207248,0.102608,-0.002110,0.011729,-0.042757,-0.067221,-0.044486,-0.010553,-0.263917,0.052020,0.154395,-0.017070,-0.051339,-0.101977,0.010399,-0.170203,0.007699,0.008053,-0.038508,-0.041094,-0.032602,0.050750,-0.029007,-0.062326,0.109653,-0.122150,0.048205,-0.060107,0.189046,0.037814,-0.037662,0.189466,-0.040528,0.032219,0.022903,-0.104137,-0.135916,0.129051,...,-0.111434,0.037832,-0.030598,-0.242025,-0.017868,-0.030671,0.103095,0.016826,0.028386,0.132345,0.261608,-0.114433,-0.114880,0.049552,-0.173208,0.001703,-0.006102,-0.136612,0.024111,0.030159,-0.077603,0.176445,0.047604,-0.032875,0.011496,0.086352,-0.122452,-0.081610,-0.084520,-0.020676,-0.824576,-0.637104,-0.014273,-0.489937,-0.712199,-0.689900,0.203428,-0.188807,-1.041112,False
2,2,-0.112302,0.123262,0.083034,-0.025925,0.002719,-0.095401,-0.076768,0.036539,0.031233,-0.231760,-0.010759,0.091659,-0.001610,-0.061019,-0.143259,-0.024492,-0.172521,0.039617,-0.028506,-0.033144,0.025336,-0.048492,0.125618,-0.069546,-0.002470,0.151607,-0.083742,0.070271,-0.038452,0.292199,0.002647,-0.160415,0.203062,-0.034336,0.050418,-0.041113,-0.063184,-0.165722,0.129476,...,-0.135445,0.071427,0.023589,-0.236292,0.030935,0.020761,0.095851,-0.036780,0.006261,0.069321,0.236453,-0.101573,-0.125460,0.112112,-0.169674,0.027782,0.048329,-0.109417,0.022625,0.057984,-0.161117,0.119863,0.028339,-0.018997,0.001445,0.091897,-0.050755,-0.029823,-0.026395,0.027364,-0.594262,-0.297496,0.703333,-0.489937,-0.174515,-0.516384,-0.503771,-1.519513,0.790146,False
3,3,-0.053385,0.214746,0.021882,0.015795,-0.019167,-0.045329,-0.057965,0.084099,0.032091,-0.239476,0.045541,0.127291,0.108474,-0.017606,-0.054341,0.038934,-0.146744,0.135011,-0.047707,0.024707,-0.048830,-0.061580,-0.035538,0.026508,-0.057374,0.039397,-0.038100,-0.037311,-0.100791,0.107844,-0.016423,-0.053083,0.109905,-0.058154,0.046641,-0.029522,-0.051385,-0.118465,0.067749,...,-0.039495,-0.071192,0.006156,-0.176517,-0.093313,-0.045471,0.055064,0.116407,0.083690,0.196422,0.159566,0.009361,-0.091297,0.105039,-0.027694,-0.093640,-0.035144,-0.142031,0.092803,0.000140,-0.208693,0.200283,0.061376,-0.039773,0.012954,-0.008263,-0.118188,-0.183112,-0.009942,-0.060312,3.167530,3.310840,0.523437,1.245600,2.245064,2.259873,0.786688,-0.413902,-2.379016,False
4,4,-0.018335,0.202566,0.023854,0.021207,0.018555,0.012194,-0.049137,-0.005782,-0.034290,-0.227361,0.048698,0.150188,-0.014524,0.057379,-0.038083,-0.131232,-0.062469,-0.040641,-0.048399,-0.122306,0.013465,0.038691,0.056038,-0.046874,0.007244,0.108148,-0.133234,0.110886,-0.031193,0.194429,0.019248,-0.083311,0.123750,-0.046843,-0.018181,-0.036381,-0.135259,0.017777,0.104087,...,-0.049712,0.039672,-0.126145,-0.

In [0]:
X = train_data.drop(columns=['Unnamed: 0', 'cdc_present'], axis=1)
y = train_data['cdc_present']

In [0]:
# Initialize SMOTE for class imbalance handling
sm = SMOTE(random_state=42)

# Split features,target into train/test sets
X_trn, X_tst, y_trn, y_tst = tts(X, y, test_size=0.25, random_state=42)

# resample train set
X_res, y_res = sm.fit_resample(X_trn, y_trn)


###Execute First Level Model

The test set will be sequestered for use in the final pipeline.

In [0]:
lr = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
lr.fit(X_res, y_res)

pn_lr = lr.predict(X_trn)
pt_lr = lr.predict(X_tst)

print("\n----------------------LR Scores----------------------")
print('Training set score:', lr.score(X_trn, y_trn))
print('Training precision:', precision_score(y_trn, pn_lr))
print('Training recall:', recall_score(y_trn, pn_lr))
print('\nTest set score:', lr.score(X_tst, y_tst))
print('Test precision:', precision_score(y_tst, pt_lr))
print('Test recall:', recall_score(y_tst, pt_lr))


----------------------LR Scores----------------------
Training set score: 0.7972634051874663
Training precision: 0.11612626767604628
Training recall: 0.8041543026706232

Test set score: 0.7958095238095239
Test precision: 0.10462602680501513
Test recall: 0.7682539682539683


In [0]:
conf_matrix(y_trn, pn_lr)

,Predicted_0,Predicted_1
Actual_0,24300,6188
Actual_1,198,813


In [0]:
conf_matrix(y_tst, pt_lr)

,Predicted_0,Predicted_1
Actual_0,8114,2071
Actual_1,73,242


## Prepare for Filter 2

In [0]:
# All Candidate Sentences are the predicted class 1
# Select all candidates for next level
X_trn['pred'] = pn_lr

idx = X_trn[X_trn['pred'] == True].index

rdf = df[['cdc_present', 'context', 'clause_0', 'clause_1', 'clause_2', 
         'clause_3', 'clause_4']].iloc[idx]

rdf.head()

,cdc_present,context,clause_0,clause_1,clause_2,clause_3,clause_4
29636,False,endangered species,the relationship between temporal variations i...,.,.,.,.
22602,True,multiculturalism,"He argued that ""the evidence is clear that man...","""we should think very carefully about the peri...",.,.,.
32710,False,atheism,belief that God has no interest in humanity an...,.,.,.,.
13298,False,nuclear weapons,in combination with US attempts to deny Iran ...,may destroy the NPT regime,while others contend that such a move will li...,.,.
26554,False,right to bear arms,limiting gun ownership to the very wealthy,"expressed the hope that Americans ""never cease...",.,.,.


In [0]:
rdf.reset_index(drop=True, inplace=True)

In [0]:
# Select all Candidates from test set
test_idx = X_tst.index

In [0]:
tdf = df.iloc[test_idx]
tdf.head()

,text,topic,article,cdc,cdc_present,context,text_words,text_syllables,text_sentiment_words,text_punctuation,text_nouns,text_phrases,sent_rat,clauses,num_clauses,claused_sentence,context_similarity,expanded_similarity,subjectivity_score,clause_0,clause_1,clause_2,clause_3,clause_4
7497,Political criticisms pose an alternative conce...,the sale of violent video games to minors,Media influence,---,False,violent video games,31,65,4,3,8,12,0.129032,[' critical subjects who are alert to genre no...,1,critical subjects who are alert to genre norm...,0.327442,0.302523,-0.0258,critical subjects who are alert to genre norm...,.,.,.,.
30459,"Joe Walston, director of the Wildlife Conserva...",endangered species should be protected,Biodiversity,---,False,endangered species,22,40,1,5,6,9,0.045455,"['Joe Walston, director of the Wildlife Conser...",1,"Joe Walston, director of the Wildlife Conserva...",0.304763,0.453497,-0.5267,"Joe Walston, director of the Wildlife Conserva...",.,.,.,.
25533,Most of the firearms violence in Mexico is per...,limit the right to bear arms,Gun violence and gun control in Texas,---,False,right to bear arms,36,67,5,3,11,18,0.138889,['drug trafficking organizations (DTOs) who ar...,1,drug trafficking organizations (DTOs) who are ...,0.354804,0.320510,0.0516,drug trafficking organizations (DTOs) who are ...,.,.,.,.
4455,The Malays viewed Independence as restoring th...,the use of affirmative action,Affirmative action,---,False,affirmative aciton,33,63,1,1,8,11,0.030303,"[""restoring their proper place in their own co...",1,restoring their proper place in their own coun...,0.139531,0.140120,0.2960,restoring their proper place in their own coun...,.,.,.,.
26693,"The court noted that it had, prior to Heller, ...",limit the right to bear arms,Second Amendment to the United States Constitu...,---,False,right to bear arms,38,66,2,6,10,13,0.052632,[' identified the Second Amendment as providin...,2,identified the Second Amendment as providing ...,0.545282,0.283945,0.3875,identified the Second Amendment as providing ...,restrictions on felon ownership of firearms di...,.,.,.


In [0]:
tdf.reset_index(drop=True, inplace=True)

### Check Grammar of Clauses

The assumption of this CDC extraction approach is that independent clauses should be able to stand alone as sentences.  That said, the minimum elements of a sentence should be present.  Additionally, they should be grammatically correct.

The process is a funnel or filter.  This step in the process is intended to further narrow down the list of candidates.

#### Train Set

In [0]:
grammar_0 = []
grammar_1 = []
grammar_2 = []
grammar_3 = []
grammar_4 = []

for x in rdf['clause_0'][:1000]:
    grammar_0.append(check_grammar(x))

In [0]:
for x in rdf['clause_0'][1000:2000]:
    grammar_0.append(check_grammar(x))

In [0]:
for x in rdf['clause_0'][2000:3000]:
    grammar_0.append(check_grammar(x))

In [0]:
for x in rdf['clause_0'][3000:4000]:
    grammar_0.append(check_grammar(x))

In [0]:
for x in rdf['clause_0'][4000:5000]:
    grammar_0.append(check_grammar(x))

In [0]:
for x in rdf['clause_0'][5000:6000]:
    grammar_0.append(check_grammar(x))

In [0]:
for x in rdf['clause_0'][6000:]:
    grammar_0.append(check_grammar(x))

In [0]:
len(grammar_0)

7001

In [0]:
for x in rdf['clause_1'][:1000]:
    grammar_1.append(check_grammar(x))

In [0]:
for x in rdf['clause_1'][1000:2000]:
    grammar_1.append(check_grammar(x))

In [0]:
for x in rdf['clause_1'][2000:1000]:
    grammar_1.append(check_grammar(x))

In [0]:
for x in rdf['clause_1'][2000:3000]:
    grammar_1.append(check_grammar(x))

In [0]:
for x in rdf['clause_1'][3000:4000]:
    grammar_1.append(check_grammar(x))

In [0]:
for x in rdf['clause_1'][4000:5000]:
    grammar_1.append(check_grammar(x))

In [0]:
for x in rdf['clause_1'][5000:6000]:
    grammar_1.append(check_grammar(x))

In [0]:
for x in rdf['clause_1'][6000:]:
    grammar_1.append(check_grammar(x))

In [0]:
len(grammar_1)

7001

In [0]:
for x in rdf['clause_2']:
    grammar_2.append(check_grammar(x))

len(grammar_2)

7001

In [0]:
for x in rdf['clause_3']:
    grammar_3.append(check_grammar(x))

len(grammar_3)

7001

In [0]:
for x in rdf['clause_4']:
    grammar_4.append(check_grammar(x))

len(grammar_4)

7001

In [0]:
rdf['grammar_0'] = grammar_0
rdf['grammar_1'] = grammar_1
rdf['grammar_2'] = grammar_2
rdf['grammar_3'] = grammar_3
rdf['grammar_4'] = grammar_4

rdf.tail(3)

,cdc_present,context,clause_0,clause_1,clause_2,clause_3,clause_4,grammar_0,grammar_1,grammar_2,grammar_3,grammar_4
6998,False,monarchy,questioning of the monarchy's role in Canadian...,.,.,.,.,True,False,False,False,False
6999,False,right to asylum,police enjoyed some success in this work but t...,job of immigration officers using powers under...,there was a clear need for the Immigration Ser...,.,.,True,True,True,False,False
7000,False,collective bargaining,"labor must ""abolish the wage system which is m...",IWW later echoed his words in its Preamble.,.,.,.,True,True,False,False,False


#### Test Set

In [0]:
test_set['pred'] = pt_lr

tidx = test_set.loc[test_set['pred'] == True].index

tdf = df.iloc[tidx]

tdf.shape

(2313, 24)

In [0]:
tdf.head().transpose()

,26693,5186,24912,13208,8040
text,"The court noted that it had, prior to Heller, ...",Friedrich Hayek felt that luck was too much of...,The only socialization that takes place explic...,The argument put forth is that not only does P...,Boys who didn’t play any video games during a ...
topic,limit the right to bear arms,the use of affirmative action,make physical education compulsory,the US is justified in using force to prevent ...,the sale of violent video games to minors
article,Second Amendment to the United States Constitu...,Equal opportunity,Democratic education,Treaty on the Non-Proliferation of Nuclear Wea...,Grand Theft Childhood
cdc,---,---,---,---,---
cdc_present,False,False,False,False,False
context,right to bear arms,affirmative aciton,physical education,nuclear weapons,violent video games
text_words,38,32,16,42,33
text_syllables,66,48,36,67,54
text_sentiment_words,2,2,0,3,4
text_punctuation,6,2,1,1,3


In [0]:
tgrammar_0 = []
tgrammar_1 = []
tgrammar_2 = []
tgrammar_3 = []
tgrammar_4 = []

for x in tdf['clause_0'][:1000]:
    tgrammar_0.append(check_grammar(x))

In [0]:
for x in tdf['clause_0'][1000:2000]:
    tgrammar_0.append(check_grammar(x))

In [0]:
for x in tdf['clause_0'][2000:]:
    tgrammar_0.append(check_grammar(x))

In [0]:
for x in tdf['clause_1'][:1000]:
    tgrammar_1.append(check_grammar(x))

In [0]:
for x in tdf['clause_1'][1000:2000]:
    tgrammar_1.append(check_grammar(x))

In [0]:
for x in tdf['clause_1'][2000:]:
    tgrammar_1.append(check_grammar(x))

In [0]:
for x in tdf['clause_2'][:1000]:
    tgrammar_2.append(check_grammar(x))

In [0]:
for x in tdf['clause_2'][1000:2000]:
    tgrammar_2.append(check_grammar(x))

In [0]:
for x in tdf['clause_2'][2000:]:
    tgrammar_2.append(check_grammar(x))

In [0]:
for x in tdf['clause_3'][:1000]:
    tgrammar_3.append(check_grammar(x))

In [0]:
for x in tdf['clause_3'][1000:2000]:
    tgrammar_3.append(check_grammar(x))

In [0]:
for x in tdf['clause_3'][2000:]:
    tgrammar_3.append(check_grammar(x))

In [0]:
for x in tdf['clause_4'][:1000]:
    tgrammar_4.append(check_grammar(x))

In [0]:
for x in tdf['clause_4'][1000:2000]:
    tgrammar_4.append(check_grammar(x))

In [0]:
for x in tdf['clause_4'][2000:]:
    tgrammar_4.append(check_grammar(x))

In [0]:
tdf['grammar_0'] = tgrammar_0
tdf['grammar_1'] = tgrammar_1
tdf['grammar_2'] = tgrammar_2
tdf['grammar_3'] = tgrammar_3
tdf['grammar_4'] = tgrammar_4

## Feature Engineering

### Training Set

In [0]:
stance_0 = []
stance_1 = []
stance_2 = []
stance_3 = []
stance_4 = []

for i,clause in enumerate(rdf['clause_0']):
    stance_0.append(calculate_stance(rdf['context'][i], clause))  

In [0]:
for i,clause in enumerate(rdf['clause_1']):
    stance_1.append(calculate_stance(rdf['context'][i], clause))  

In [0]:
for i,clause in enumerate(rdf['clause_2']):
    stance_2.append(calculate_stance(rdf['context'][i], clause))

In [0]:
for i,clause in enumerate(rdf['clause_3']):
    stance_3.append(calculate_stance(rdf['context'][i], clause))

In [0]:
for i,clause in enumerate(rdf['clause_4']):
    stance_4.append(calculate_stance(rdf['context'][i], clause))

In [0]:
rdf['stance_0'] = stance_0
rdf['stance_1'] = stance_1
rdf['stance_2'] = stance_2
rdf['stance_3'] = stance_3
rdf['stance_4'] = stance_4

rdf.tail(3)

,cdc_present,context,clause_0,clause_1,clause_2,clause_3,clause_4,grammar_0,grammar_1,grammar_2,grammar_3,grammar_4,stance_0,stance_1,stance_2,stance_3,stance_4
6998,False,monarchy,questioning of the monarchy's role in Canadian...,.,.,.,.,True,False,False,False,False,-0.036252,0.0,0.000000,0.0,0.0
6999,False,right to asylum,police enjoyed some success in this work but t...,job of immigration officers using powers under...,there was a clear need for the Immigration Ser...,.,.,True,True,True,False,False,0.230343,0.0,0.253302,0.0,0.0
7000,False,collective bargaining,"labor must ""abolish the wage system which is m...",IWW later echoed his words in its Preamble.,.,.,.,True,True,False,False,False,-0.076254,0.0,0.000000,0.0,0.0


In [0]:
rdf = trim_dataframe(rdf)

In [0]:
rdf.shape

(6589, 4)

In [0]:
rdf['cdc_present'].value_counts()

False    5818
True      771
Name: cdc_present, dtype: int64

In [0]:
rdf.head(3).transpose()

,0,1,2
cdc_present,True,False,False
context,multiculturalism,atheism,nuclear weapons
clause,"He argued that ""the evidence is clear that man...",belief that God has no interest in humanity an...,"labor must ""abolish the wage system which is m..."
stance,-0.294746,0.109423,-0.076254


### Test Set

In [0]:
tdf.reset_index(drop=True, inplace=True)

In [0]:
tstance_0 = []
tstance_1 = []
tstance_2 = []
tstance_3 = []
tstance_4 = []

for i,clause in enumerate(tdf['clause_0']):
    tstance_0.append(calculate_stance(tdf['context'][i], clause))  

In [0]:
for i,clause in enumerate(tdf['clause_1']):
    tstance_1.append(calculate_stance(tdf['context'][i], clause))  

In [0]:
for i,clause in enumerate(tdf['clause_2']):
    tstance_2.append(calculate_stance(tdf['context'][i], clause)) 

In [0]:
for i,clause in enumerate(tdf['clause_3']):
    tstance_3.append(calculate_stance(tdf['context'][i], clause)) 

In [0]:
for i,clause in enumerate(tdf['clause_4']):
    tstance_4.append(calculate_stance(tdf['context'][i], clause)) 

In [0]:
tdf['stance_0'] = tstance_0
tdf['stance_1'] = tstance_1
tdf['stance_2'] = tstance_2
tdf['stance_3'] = tstance_3
tdf['stance_4'] = tstance_4

In [0]:
tdf.head().transpose()

,0,1,2,3,4
text,"The court noted that it had, prior to Heller, ...",Friedrich Hayek felt that luck was too much of...,The only socialization that takes place explic...,The argument put forth is that not only does P...,Boys who didn’t play any video games during a ...
topic,limit the right to bear arms,the use of affirmative action,make physical education compulsory,the US is justified in using force to prevent ...,the sale of violent video games to minors
article,Second Amendment to the United States Constitu...,Equal opportunity,Democratic education,Treaty on the Non-Proliferation of Nuclear Wea...,Grand Theft Childhood
cdc,---,---,---,---,---
cdc_present,False,False,False,False,False
context,right to bear arms,affirmative aciton,physical education,nuclear weapons,violent video games
text_words,38,32,16,42,33
text_syllables,66,48,36,67,54
text_sentiment_words,2,2,0,3,4
text_punctuation,6,2,1,1,3


In [0]:
tdf = trim_test_dataframe(tdf, cols=['cdc', 'cdc_present', 'context', 'clause', 
                                     'stance'])

In [0]:
tdf['cdc_present'].value_counts()

False    1957
True      219
Name: cdc_present, dtype: int64

## Extract and organize Metadata


#### Train Set

In [0]:
# Exctract information from text for evaluation
rdf['words'] = rdf['clause'].apply(lambda x: word_count(x))
rdf['syllables'] = rdf['clause'].apply(lambda x: count_syllables(x))
rdf['sentiment_words'] = rdf['clause'].apply(lambda x: count_sentiment_words(x))
rdf['nouns'] = rdf['clause'].apply(lambda x: noun_counts(x))

In [0]:
rdf['sent_rat'] = rdf['sentiment_words'] / rdf['words']

In [0]:
# Compute context similarity of each sentence
context_similarity = []

for i,txt in enumerate(rdf['clause']):
    context_similarity.append(subject_match(rdf['context'][i], txt))

rdf['context_similarity'] = context_similarity

In [0]:
# Compute similarity of sentence with expanded context
expanded_similarity = []
for i,txt in enumerate(rdf['clause']):
    expanded_similarity.append(expanded_sentence_similarity(rdf['context'][i], txt))

rdf['expanded_similarity'] = expanded_similarity

In [0]:
rdf[['context_similarity', 'expanded_similarity', 'cdc_present']].head(10)

,context_similarity,expanded_similarity,cdc_present
0,0.214705,0.253651,True
1,0.413659,0.210921,False
2,0.327303,0.210071,False
3,0.619828,0.209294,False
4,0.541051,0.374933,False
5,0.401344,0.537170,False
6,0.244186,0.436574,False
7,0.415510,0.388887,False
8,0.415609,0.386135,False
9,0.666958,0.236793,False


In [0]:
rdf['subjectivity_score'] = rdf['clause'].apply(lambda x: sentiment_subjectivity(x, 'subjectivity'))

In [0]:
rdf.head().transpose()

,0,1,2,3,4
cdc_present,True,False,False,False,False
context,multiculturalism,atheism,nuclear weapons,right to bear arms,one child policy
clause,"He argued that ""the evidence is clear that man...",belief that God has no interest in humanity an...,"labor must ""abolish the wage system which is m...","expressed the hope that Americans ""never cease...",and that some people consider themselves memb...
stance,-0.294746,0.109423,-0.076254,0.327691,-0.164083
words,24,16,21,23,18
syllables,37,27,35,35,28
sentiment_words,4,3,2,3,1
nouns,5,5,5,6,4
sent_rat,0.166667,0.1875,0.0952381,0.130435,0.0555556
context_similarity,0.214705,0.413659,0.327303,0.619828,0.541051


In [0]:
cxt = [nlp(context) for context in rdf['context']]
v_cxt = [ct.vector for ct in cxt]

In [0]:
txt = [nlp(clause) for clause in rdf['clause']]
v_txt = [t.vector for t in txt]

In [0]:
txt_vects = pd.DataFrame(v_txt)
txt_vects.columns = ['txt_'+str(col) for col in txt_vects.columns]

In [0]:
cxt_vects = pd.DataFrame(v_cxt)
cxt_vects.columns = ['cxt_'+str(col) for col in cxt_vects.columns]

In [0]:
rdf = pd.concat([rdf, txt_vects, cxt_vects], axis=1)

In [0]:
rdf.head().transpose()

,0,1,2,3,4
cdc_present,True,False,False,False,False
context,multiculturalism,atheism,nuclear weapons,right to bear arms,one child policy
clause,"He argued that ""the evidence is clear that man...",belief that God has no interest in humanity an...,"labor must ""abolish the wage system which is m...","expressed the hope that Americans ""never cease...",and that some people consider themselves memb...
stance,-0.294746,0.109423,-0.076254,0.327691,-0.164083
words,24,16,21,23,18
...,...,...,...,...,...
cxt_295,0.57975,0.075376,0.252915,-0.141105,0.0461523
cxt_296,-0.30301,0.38698,-0.30077,0.0856638,-0.234906
cxt_297,0.039072,0.20169,0.258464,0.0151028,-0.0799367
cxt_298,0.21212,-0.071851,0.025537,-0.0143727,0.226938


In [0]:
null = rdf.isna()

ndx = null[null['sent_rat'] == True].index

rdf.iloc[ndx]

,cdc_present,context,clause,stance,words,syllables,sentiment_words,nouns,sent_rat,context_similarity,expanded_similarity,subjectivity_score,txt_0,txt_1,txt_2,txt_3,txt_4,txt_5,txt_6,txt_7,txt_8,txt_9,txt_10,txt_11,txt_12,txt_13,txt_14,txt_15,txt_16,txt_17,txt_18,txt_19,txt_20,txt_21,txt_22,txt_23,txt_24,txt_25,txt_26,txt_27,...,cxt_260,cxt_261,cxt_262,cxt_263,cxt_264,cxt_265,cxt_266,cxt_267,cxt_268,cxt_269,cxt_270,cxt_271,cxt_272,cxt_273,cxt_274,cxt_275,cxt_276,cxt_277,cxt_278,cxt_279,cxt_280,cxt_281,cxt_282,cxt_283,cxt_284,cxt_285,cxt_286,cxt_287,cxt_288,cxt_289,cxt_290,cxt_291,cxt_292,cxt_293,cxt_294,cxt_295,cxt_296,cxt_297,cxt_298,cxt_299
61,False,violent video games,.,0.0,0,1,0,0,NaN,NaN,0.413451,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,0.035603,...,0.271563,0.135767,-0.191623,0.033933,0.006038,-0.208417,-0.039990,0.104643,0.368580,-0.047727,-0.463380,-0.204360,0.048473,-0.205093,-0.363393,0.033035,-0.010813,-0.022343,-0.075073,0.207637,0.337060,0.113085,-0.186804,-0.026109,-0.091814,0.132277,-0.041427,-0.309250,0.321133,-0.391453,-0.356706,-0.081472,-0.350020,0.164247,0.399030,0.239813,-0.153626,-0.230494,-0.117433,0.191165
166,False,right to asylum,.,0.0,0,1,0,0,NaN,NaN,0.393511,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,0.035603,...,-0.286717,0.207575,0.178091,-0.060810,0.195185,-0.219669,-0.181153,-0.008457,0.242420,-0.067078,0.314793,-0.029574,0.016692,-0.237093,-0.019864,-0.038480,0.187315,0.097803,0.055328,0.215803,0.069756,0.252599,0.043200,0.099307,-0.228413,-0.114273,-0.179890,0.238907,0.176899,-0.266001,-0.190500,0.052705,0.140644,-0.283945,0.206423,-0.265083,-0.056204,0.112758,-0.012630,0.209857
627,False,austerity measures,.,0.0,0,1,0,0,NaN,NaN,0.338342,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,0.035603,...,-0.056395,0.528875,-0.137628,0.233905,-0.392957,0.002285,-0.224455,-0.072547,0.276756,-0.030397,0.572780,0.052540,-0.308745,-0.091502,0.392610,0.340840,0.683425,0.379645,0.235475,0.219738,0.438150,-0.106280,0.207547,0.386710,0.024826,-0.157651,-0.046985,0.134404,-0.242321,0.095336,-0.091245,0.578930,0.619055,0.525030,0.454925,-0.026794,0.045230,-0.151260,0.459130,-0.178115
1123,False,wind power,.,0.0,0,1,0,0,NaN,NaN,0.338342,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,0.035603,...,0.163440,0.075791,0.259845,0.202452,0.608575,-0.442635,-0.315780,0.430770,0.285535,-0.141530,0.419085,0.048265,-0.377010,0.190910,0.154525,0.121289,0.266880,0.099545,0.055127,-0.141215,-0.252110,-0.260044,0.119079,0.552145,0.032605,0.444400,0.418880,-0.322270,0.357400,-0.002759,-0.540460,0.014123,0.426770,-0.453605,-0.214370,0.313740,0.170541,0.261233,-0.120150,-0.136557
1746,True,affirmative aciton,.,0.0,0,1,0,0,NaN,NaN,-0.086151,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,0.035603,...,-0.899770,0.074667,0.011280,0.078425,-0.303396,0.275712,0.096043,-0.110640,0.257587,0.352364,0.183540,-0.294305,-0.195535,-0.088610,0.339498,0.434715,-0.214650,0.280800,0.512500,-0.113770,-0.219930,-0.529584,-0.050285,-0.094307,-0.067860,-0.233844,-0.218420,-0.088710,-0.048320,-0.172400,-0.020990,0.002080,

In [0]:
rdf.dropna(inplace=True)

In [0]:
write_path = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/funnel_2.csv'

rdf.to_csv(write_path)

#### Test Set

In [0]:
# Exctract information from text for evaluation
tdf['words'] = tdf['clause'].apply(lambda x: word_count(x))
tdf['syllables'] = tdf['clause'].apply(lambda x: count_syllables(x))
tdf['sentiment_words'] = tdf['clause'].apply(lambda x: count_sentiment_words(x))
tdf['nouns'] = tdf['clause'].apply(lambda x: noun_counts(x))

In [0]:
tdf['sent_rat'] = tdf['sentiment_words'] / tdf['words']

In [0]:
# Compute context similarity of each sentence
context_similarity = []

for i,txt in enumerate(tdf['clause']):
    context_similarity.append(subject_match(tdf['context'][i], txt))

tdf['context_similarity'] = context_similarity

In [0]:
# Compute similarity of sentence with expanded context
expanded_similarity = []
for i,txt in enumerate(tdf['clause']):
    expanded_similarity.append(expanded_sentence_similarity(tdf['context'][i], txt))

tdf['expanded_similarity'] = expanded_similarity

In [0]:
tdf['subjectivity_score'] = tdf['clause'].apply(lambda x: sentiment_subjectivity(x, 'subjectivity'))

In [0]:
cxt = [nlp(context) for context in tdf['context']]
v_cxt = [ct.vector for ct in cxt]

In [0]:
txt = [nlp(clause) for clause in tdf['clause']]
v_txt = [t.vector for t in txt]

In [0]:
txt_vects = pd.DataFrame(v_txt)
txt_vects.columns = ['txt_'+str(col) for col in txt_vects.columns]

In [0]:
cxt_vects = pd.DataFrame(v_cxt)
cxt_vects.columns = ['cxt_'+str(col) for col in cxt_vects.columns]

In [0]:
tdf = pd.concat([tdf, txt_vects, cxt_vects], axis=1)

In [0]:
tdf.head().transpose()

,0,1,2,3,4
cdc,---,---,---,---,---
cdc_present,False,False,False,False,False
context,right to bear arms,affirmative aciton,physical education,nuclear weapons,violent video games
clause,restrictions on felon ownership of firearms di...,luck was too much of a variable in economics,only socialization that takes place explicitly,not only does Pakistan lack the same energy re...,Boys who didn’t play any video games during a ...
stance,0.106768,0.111407,0,-0.00962334,-0.0147226
...,...,...,...,...,...
cxt_295,-0.141105,-0.0966485,0.2646,0.252915,0.239813
cxt_296,0.0856638,-0.187825,0.10351,-0.30077,-0.153626
cxt_297,0.0151028,0.461215,-0.20979,0.258464,-0.230494
cxt_298,-0.0143727,0.214574,-0.153765,0.025537,-0.117433


In [0]:
write_path = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/m2_test_set.csv'

tdf.to_csv(write_path)

In [0]:
m1_test = pd.concat([X_tst,y_tst])

In [0]:
write_path = '/content/drive/My Drive/Colab Notebooks/Thinkful/Module 34 - Final Capstone/data/CDC Detection/m1_test_set.csv'

m1_test.to_csv(write_path)

In [0]:
null = tdf.isna().sum()

null[null>0]

sent_rat              5
context_similarity    5
dtype: int64

In [0]:
null = tdf.isna()

ndx = null[null['sent_rat']==True].index

tdf.iloc[ndx]

,cdc,cdc_present,context,clause,stance,words,syllables,sentiment_words,nouns,sent_rat,context_similarity,expanded_similarity,subjectivity_score,txt_0,txt_1,txt_2,txt_3,txt_4,txt_5,txt_6,txt_7,txt_8,txt_9,txt_10,txt_11,txt_12,txt_13,txt_14,txt_15,txt_16,txt_17,txt_18,txt_19,txt_20,txt_21,txt_22,txt_23,txt_24,txt_25,txt_26,...,cxt_260,cxt_261,cxt_262,cxt_263,cxt_264,cxt_265,cxt_266,cxt_267,cxt_268,cxt_269,cxt_270,cxt_271,cxt_272,cxt_273,cxt_274,cxt_275,cxt_276,cxt_277,cxt_278,cxt_279,cxt_280,cxt_281,cxt_282,cxt_283,cxt_284,cxt_285,cxt_286,cxt_287,cxt_288,cxt_289,cxt_290,cxt_291,cxt_292,cxt_293,cxt_294,cxt_295,cxt_296,cxt_297,cxt_298,cxt_299
1161,---,False,austerity measures,.,0.0,0,1,0,0,NaN,NaN,0.338342,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,...,-0.056395,0.528875,-0.137628,0.233905,-0.392957,0.002285,-0.224455,-0.072547,0.276756,-0.030397,0.572780,0.052540,-0.308745,-0.091502,0.392610,0.340840,0.683425,0.379645,0.235475,0.219738,0.438150,-0.106280,0.207547,0.386710,0.024826,-0.157651,-0.046985,0.134404,-0.242321,0.095336,-0.091245,0.578930,0.619055,0.525030,0.454925,-0.026794,0.045230,-0.151260,0.459130,-0.178115
1284,---,False,performing enhancing drugs,.,0.0,0,1,0,0,NaN,NaN,0.362591,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,...,-0.116343,0.077119,0.106827,-0.018709,-0.248797,0.217163,0.179638,0.008774,0.456253,-0.130407,-0.130286,-0.115123,-0.324263,0.046184,0.384258,-0.055326,0.008533,-0.210150,-0.154540,0.395963,-0.174148,0.083203,0.120418,-0.083333,-0.179965,-0.143307,0.143532,-0.175579,0.148507,0.099953,-0.059063,-0.095114,-0.019135,0.017315,-0.361130,0.048481,0.138179,-0.104404,0.099953,0.435738
1332,those that give preference to individuals sole...,True,affirmative aciton,.,0.0,0,1,0,0,NaN,NaN,-0.086151,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,...,-0.899770,0.074667,0.011280,0.078425,-0.303396,0.275712,0.096043,-0.110640,0.257587,0.352364,0.183540,-0.294305,-0.195535,-0.088610,0.339498,0.434715,-0.214650,0.280800,0.512500,-0.113770,-0.219930,-0.529584,-0.050285,-0.094307,-0.067860,-0.233844,-0.218420,-0.088710,-0.048320,-0.172400,-0.020990,0.002080,-0.000328,-0.042084,0.419700,-0.096648,-0.187825,0.461215,0.214574,-0.054802
1739,---,False,bribery,.,0.0,0,1,0,0,NaN,NaN,0.338342,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,...,0.115130,-0.273770,-0.063913,0.512460,-0.424110,0.494560,0.090735,0.079770,0.228980,-0.875210,0.158690,-0.016537,0.127120,0.326440,0.402470,0.557880,0.077808,0.363350,-0.087982,-0.195540,0.169740,-0.328850,-0.071546,0.139520,0.614570,0.400330,-0.362680,0.440480,-0.226870,-0.206370,-0.216740,0.248080,0.204380,0.094798,-0.585850,0.352630,0.701680,0.361340,0.099988,0.717760
1909,---,False,nuclear weapons,.,0.0,0,1,0,0,NaN,NaN,0.338342,0.0,0.012001,0.20751,-0.12578,-0.59325,0.12525,0.15975,0.13748,-0.33157,-0.13694,1.7893,-0.47094,0.70434,0.26673,-0.089961,-0.18168,0.067226,0.053347,1.5595,-0.2541,0.038413,-0.01409,0.056774,0.023434,0.024042,0.31703,0.19025,-0.37505,...,-0.475815,0.347093,0.136070,0.362575,-0.532715,-0.179435,0.013887,0.153185,0.464345,-0.004274,0.787985,0.175290,0.321571,-0.424735,0.084160,0.112082,0.256910,0.135640,-0.050795,0.017555,0.488375,-0.351045,-0.189045,-0.467040,-0.004800,0.163235,-0.140632,-0.051086,0.185475,0.4311

In [0]:
tdf.dropna(inplace=True)